# ALL TABLES IN ONE WORKSHEET, TITLE AND HEAD ARE SEPARATED ONE LINE FROM TABLE

Dynamic PDF and Excel Paths – The script asks for the PDF path and the desired Excel filename.
Excel Creation in Same Folder – The Excel file is created in the same directory as the PDF.
Custom Excel Naming – The user can specify the Excel file name without hardcoding it.

# REMEMBER 

TO WORK WITH MORE THAN ONE PDF, THE STRUCTURE OF ALL PDFS MUST BE THE SAME OR YOU HAVE TO CREATE ONE EXCEL FILE FOR EACH PDF.

In [8]:
import pdfplumber
import pandas as pd
import os
from openpyxl import load_workbook
from tkinter import Tk, filedialog

# Open file dialog to select PDF
def get_pdf_file():
    Tk().withdraw()  # Hide the root window
    file_path = filedialog.askopenfilename(
        title="Select PDF File",
        filetypes=[("PDF Files", "*.pdf")]
    )
    return file_path

# Get the PDF file path using file dialog
pdf_file = get_pdf_file()

# Exit if no file is selected
if not pdf_file:
    print("No file selected. Exiting.")
    exit()

# Extract the directory and name from the PDF path
pdf_dir = os.path.dirname(pdf_file)

# Ask for the desired Excel file name
excel_name = input("Enter the desired name for the Excel file (without extension): ")
excel_file = os.path.join(pdf_dir, f"{excel_name}.xlsx")

# List to store tables and their titles
all_tables = []

# Function to clean spaces and irrelevant characters
def clean_string(s):
    if isinstance(s, str):
        return s.strip()
    return s

# Function to check if the title is duplicated in the table header
def remove_title_from_table(title, table):
    if len(table) > 0 and isinstance(table[0][0], str) and title == table[0][0].strip():
        return table[1:]
    return table

# Read and extract tables and titles from the PDF
with pdfplumber.open(pdf_file) as pdf:
    for i, page in enumerate(pdf.pages):
        text = page.extract_text()
        lines = text.split("\n")

        tables = page.extract_tables()
        for table in tables:
            if not table or len(table[0]) < 2:
                continue

            title = None
            for line in lines:
                if line.strip():
                    title = clean_string(line)
                    break

            table = remove_title_from_table(title, table)
            df = pd.DataFrame(table)

            df.columns = df.iloc[0]
            df = df[1:]

            df = df.map(lambda x: clean_string(x) if isinstance(x, str) else x)

            all_tables.append((title, df))

# Check if the Excel file already exists
if os.path.exists(excel_file):
    print(f"The file '{excel_file}' already exists.")
    add_new_sheet = input("Do you want to add a new worksheet (y/n)? ").lower()

    if add_new_sheet == 'y':
        book = load_workbook(excel_file)
        sheet_name = input("Enter the name of the new worksheet: ")

        worksheet = book.create_sheet(sheet_name)

        current_col = 1
        for i, (title, table) in enumerate(all_tables):
            worksheet.cell(row=1, column=current_col).value = title
            for col_idx, column in enumerate(table.columns):
                worksheet.cell(row=2, column=current_col + col_idx).value = column
            for row_idx, row in table.iterrows():
                for col_idx, value in enumerate(row):
                    worksheet.cell(row=row_idx + 2, column=current_col + col_idx).value = value
            current_col += len(table.columns) + 2

        book.save(excel_file)
        print(f"Tables saved in '{excel_file}' under worksheet '{sheet_name}'.")

else:
    # Ask for the first worksheet name when creating a new Excel file
    first_sheet_name = input("What is the name of the first worksheet? ")

    with pd.ExcelWriter(excel_file, engine="openpyxl") as writer:
        worksheet = writer.book.create_sheet(title=first_sheet_name)

        current_col = 1
        for i, (title, table) in enumerate(all_tables):
            worksheet.cell(row=1, column=current_col).value = title
            for col_idx, column in enumerate(table.columns):
                worksheet.cell(row=2, column=current_col + col_idx).value = column
            for row_idx, row in table.iterrows():
                for col_idx, value in enumerate(row):
                    worksheet.cell(row=row_idx + 3, column=current_col + col_idx).value = value
            current_col += len(table.columns) + 2

    print(f"Tables extracted and saved in '{excel_file}' with first worksheet '{first_sheet_name}'.")


Enter the desired name for the Excel file (without extension):  TEST1


The file 'E:/EEG_notebook\TEST1.xlsx' already exists.


Do you want to add a new worksheet (y/n)?  Y
Enter the name of the new worksheet:  P3


Tables saved in 'E:/EEG_notebook\TEST1.xlsx' under worksheet 'P3'.
